# Part 1

Import some of the libraries used

In [20]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

Get the Wiki page and parse it with BeautifulSoup

In [21]:
wiki_toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content, 'lxml')

Extract postal codes, boroughs and neighborhoods and put them in a DataFrame

In [29]:
def clean_content(content):
    content = content.replace(' / ', ', ')
    content = content.replace('(', '')
    content = content.replace(')', '')
    return content

items = []
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    parts = [it for it in item.stripped_strings if it not in "()"]
    borough = parts[1]
    neighborhood = ' '.join(parts[2:])
    
    items.append((postal_code, clean_content(borough), clean_content(neighborhood)))

df_toronto = pd.DataFrame([it for it in items], columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [30]:
df_toronto.shape

(103, 3)

# Part 2

In [31]:
# The code was removed by DSX for sharing.

Get location coordinates for all Postal Codes

In [32]:
locations = []

for item in df_toronto['PostalCode']:
        
    # construct URL to make API call
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address="Toronto, ON {}"'.format(API_key, item)

    response = requests.get(url).json() # get response
    
    if response['status'] == 'ZERO_RESULTS':
        print(item)
        continue
    
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    locations.append((item, latitude, longitude))
    
df_locations = pd.DataFrame([it for it in locations], columns=['PostalCode', 'Latitude', 'Longitude'])
df_locations.tail()

,PostalCode,Latitude,Longitude
98,M8X,43.653654,-79.506944
99,M4Y,43.665860,-79.383160
100,M7Y,43.662744,-79.321558
101,M8Y,43.636258,-79.498509
102,M8Z,43.628841,-79.520999


Merge coordinates with the rest of the data

In [34]:
df_data = df_toronto.merge(df_locations, on="PostalCode", how='left')

df_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


In [35]:
# The code was removed by DSX for sharing.

Use foursquare API to retrieve venues in neighborhoods

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    total = len(names)
    count = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print('Unable to fetch: {0}'.format(name))
            continue

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        count += 1
        print("{0}/{1}".format(count, total))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(df_data['Neighborhood'], df_data['Latitude'], df_data['Longitude'])

Parkwoods
1/103
Victoria Village
2/103
Regent Park, Harbourfront
3/103
Lawrence Manor, Lawrence Heights
4/103
Ontario Provincial Government
5/103
Islington Avenue
6/103
Malvern, Rouge
7/103
Don Mills North
8/103
Parkview Hill, Woodbine Gardens
9/103
Garden District , Ryerson
10/103
Glencairn
11/103
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
12/103
Rouge Hill, Port Union, Highland Creek
13/103
Don Mills South Flemingdon Park
14/103
Woodbine Heights
15/103
St. James Town
16/103
Humewood-Cedarvale
17/103
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
18/103
Guildwood, Morningside, West Hill
19/103
The Beaches
20/103
Berczy Park
21/103
Caledonia-Fairbanks
22/103
Woburn
23/103
Leaside
24/103
Central Bay Street
25/103
Christie
26/103
Cedarbrae
27/103
Hillcrest Village
28/103
Bathurst Manor, Wilson Heights, Downsview North
29/103
Thorncliffe Park
30/103
Richmond, Adelaide, King
31/103
Dufferin, Dovercourt Village
32/103
Scarborough Village
33/103
Fai

Show first venues

In [38]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Save the dataframes to storage for later usage

In [42]:
# The code was removed by DSX for sharing.

In [43]:
project.save_data('df_data.p', pickle.dumps(df_data), overwrite=True)
project.save_data('toronto_venues.p', pickle.dumps(toronto_venues), overwrite=True)
project.save_data('df_locations.p', pickle.dumps(df_locations), overwrite=True)
project.save_data('df_toronto.p', pickle.dumps(df_toronto), overwrite=True)

{'asset_id': '4420dbc1-4e6d-4b6e-958f-22753043ee6b',
 'bucket_name': 'courseracapstone-donotdelete-pr-duvaojegpzermh',
 'file_name': 'df_toronto.p',
 'message': 'File df_toronto.p has been written successfully to the associated OS'}

Load the data from the storage (work can be resumed from here)

In [44]:
toronto_venues_file = project.get_file('toronto_venues.p')
toronto_venues = pickle.load(toronto_venues_file)

df_data = pickle.load(project.get_file('df_data.p'))

In [45]:
print(toronto_venues.head())

       Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1                     KFC       43.754387       -79.333021   
2           Variety Store       43.751974       -79.333114   
3  Victoria Village Arena       43.723481       -79.315635   
4             Tim Hortons       43.725517       -79.313103   

         Venue Category  
0                  Park  
1  Fast Food Restaurant  
2     Food & Drink Shop  
3          Hockey Arena  
4           Coffee Shop  


This is how only neighborhoods containing 'Toronto' would be selecte [I decided not to filter]

In [46]:
# toronto_venues_filtered = toronto_venues[toronto_venues['Neighborhood'].str.contains('Toronto')]

Print number of venues categories

In [47]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


Encode "Venue Category" (categorical data) as onehot vectors

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Show occurence frequency for each category in each neighborhood

In [50]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Print the top 5 venue categories per neighborhood

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Clothing Store  0.25
1          Lounge  0.25
2    Skating Rink  0.25
3  Breakfast Spot  0.25
4     Yoga Studio  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.25
1      Sandwich Place  0.12
2                Pool  0.12
3  Athletics & Sports  0.12
4         Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.10
1        Pharmacy  0.05
2  Ice Cream Shop  0.05
3   Shopping Mall  0.05
4      Restaurant  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.07
1    Italian Restaurant  0.07
2      Sushi Restaurant  0.07
3             Juice Bar  0.07
4  Fast Food Restaurant  0.07


----Bercz

Display top 10 venues for each neighborhood

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner
1,"Alderwood, Long Branch",Pizza Place,Bank,Sandwich Place,Pub,Pool,Athletics & Sports,Coffee Shop,Diner,Dim Sum Restaurant,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Pizza Place,Restaurant,Bank,Bridal Shop,Supermarket,Diner,Sushi Restaurant,Pharmacy,Pet Store
3,Bayview Village,Chinese Restaurant,Japanese Restaurant,Bank,Café,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Juice Bar,Sushi Restaurant,Greek Restaurant,Hardware Store,Sandwich Place,Restaurant,Japanese Restaurant


In [54]:
!pip install geopy folium

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/e

Run *k*-means to cluster the neighborhood into 5 clusters.

In [55]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
toronto_merged = toronto_grouped

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Pizza Place,Bank,Sandwich Place,Pub,Pool,Athletics & Sports,Coffee Shop,Diner,Dim Sum Restaurant,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Pizza Place,Restaurant,Bank,Bridal Shop,Supermarket,Diner,Sushi Restaurant,Pharmacy,Pet Store
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Chinese Restaurant,Japanese Restaurant,Bank,Café,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Juice Bar,Sushi Restaurant,Greek Restaurant,Hardware Store,Sandwich Place,Restaurant,Japanese Restaurant


Add the localization coordinates to the new dataframe

In [57]:
localization_data = df_data[['Neighborhood', 'Latitude', 'Longitude']]
localization_data.set_index('Neighborhood', inplace=True)
toronto_merged = toronto_merged.join(localization_data, on='Neighborhood')
toronto_merged.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Breakfast Spot,Clothing Store,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner,43.794200,-79.262029
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Sandwich Place,Pub,Pool,Athletics & Sports,Coffee Shop,Diner,Dim Sum Restaurant,Discount Store,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Restaurant,Bank,Bridal Shop,Supermarket,Diner,Sushi Restaurant,Pharmacy,Pet Store,43.754328,-79.442259
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Bank,Café,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Fast Food Restaurant,Juice Bar,Sushi Restaurant,Greek Restaurant,Hardware Store,Sandwich Place,Restaurant,Japanese Restaurant,43.733283,-79.419750


Show clustering on map

In [58]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Looking at the two neighborhoods that belong to Cluster 0 show that they are pretty similar in terms of "most common venue"

In [59]:
toronto_merged[toronto_merged['Cluster Labels'] == 0].filter(regex='Neighborhood|Most Common Venue')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Women's Store,Curling Ice,Drugstore,Dog Run,Discount Store,Diner
1,"Alderwood, Long Branch",Pizza Place,Bank,Sandwich Place,Pub,Pool,Athletics & Sports,Coffee Shop,Diner,Dim Sum Restaurant,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Pizza Place,Restaurant,Bank,Bridal Shop,Supermarket,Diner,Sushi Restaurant,Pharmacy,Pet Store
3,Bayview Village,Chinese Restaurant,Japanese Restaurant,Bank,Café,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Juice Bar,Sushi Restaurant,Greek Restaurant,Hardware Store,Sandwich Place,Restaurant,Japanese Restaurant
5,Berczy Park,Beer Bar,Seafood Restaurant,Bakery,Farmers Market,Cocktail Bar,Clothing Store,Comfort Food Restaurant,Concert Hall,Park,Café
6,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,College Stadium,Café,Cuban Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
7,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Performing Arts Venue,Café,Breakfast Spot,Burrito Place,Climbing Gym,Grocery Store,Gym,Gym / Fitness Center,Falafel Restaurant
8,Business reply mail Processing Centre 969 East...,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Burrito Place
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Plane,Sculpture Garden
